# import packages

In [22]:
# more common imports
import pandas as pd
import numpy as np
from collections import Counter
import re

# languange processing imports
import nltk
import string
from gensim.corpora import Dictionary
# preprocessing imports
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
# model imports
from gensim.utils import simple_preprocess

from gensim.models import CoherenceModel, Phrases, phrases
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from sklearn.preprocessing import normalize

import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)

# read tables from mysql

In [ ]:
printable = set(string.printable)
s = 'Ti\u00ebsto'
list(filter(lambda x: x in printable, s))

In [57]:
# read lyrics.csv
data = pd.read_csv('lyrics.csv', names=['singer', 'song', 'lyrics'])

In [58]:
def preprocess(text):
    try:
        return simple_preprocess(text)
    except Exception as e:
        print(str(e))
        return np.nan

In [ ]:
#lyrics['singer'] = list(map(simple_preprocess, lyrics.singer))
#lyrics['song'] = list(map(simple_preprocess, lyrics.singer))
data['lyrics'] = list(map(preprocess, data.lyrics.values))
data = data[~data['lyrics'].isna()]

# constrcut doc2vec model

In [18]:
def infer_w2v_features(d2v_model, sentence):
    #vector = d2v_model.docvecs[i]
        
    inferred_vector = d2v_model.infer_vector(sentence)
    return inferred_vector

In [ ]:
from gensim.test.utils import get_tmpfile
train_corpus = list(map(lambda i:TaggedDocument(data.iloc[i].lyrics,['doc%d' % i]), range(0,data.shape[0])))

model = Doc2Vec(min_count=1, window=10, vector_size=100, sample=1e-4, negative=5, workers=8, epochs=200)
model.build_vocab(train_corpus)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

#train_data3['w2v_features'] = list(map(lambda i:
#                                      get_w2v_features(model, i),
#                                      range(0, train_data3.shape[0])))
fname = get_tmpfile("my_doc2vec_model")

model.save(fname)


In [ ]:
fname = ge

In [20]:
ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index('doc%d' % doc_id)
    ranks.append(rank)
    
import collections
collections.Counter(ranks) 
    

Counter({0: 6872, 1: 3})

In [21]:
import pickle
with open('store_ranks.npy', 'wb') as writer:
    pickle.dump(ranks, writer)
    
with open('store_ranks.npy', 'rb') as reader:
    ranks3 = pickle.load(reader)
collections.Counter(ranks3)

Counter({0: 6872, 1: 3})

In [22]:
#model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
train_data3['w2v_features'] = list(map(lambda sentence:
                                      infer_w2v_features(model, sentence),
                                      train_data3.tokenized_sentences.values))
test_data3['w2v_features'] = list(map(lambda sentence:
                                      infer_w2v_features(model, sentence),
                                      test_data3.tokenized_sentences.values))

In [29]:
index = np.concatenate((train_data3.id.values, test_data3.id.values))
columns = test_data3.columns.values[10:-2]
reviewers_semantic_sim = pd.DataFrame(index=index, columns=columns)
dataset = pd.concat([train_data3, test_data3])

In [30]:
for index, pr in dataset.iterrows():
    for reviewer in train_data3.columns.values[10:-2]:
        train_data = dataset.loc[:index][:-1]
        reviewed_prs = train_data[train_data[reviewer]==1]
        if reviewed_prs.shape[0] == 0:
            reviewers_semantic_sim.loc[pr.id, reviewer] = [-1]
            continue
        list_vectors = list(reviewed_prs.w2v_features.values)
        ##vector_features = normalize(pr.w2v_features[:,np.newaxis], axis=0).ravel() 
        ##similar_scores = [np.dot(normalize(pr_vector[:,np.newaxis], axis=0).ravel(), vector_features) for pr_vector in reviewers_vectors[reviewer]]
        similar_scores = [np.dot(pr_vector, pr.w2v_features)/(np.linalg.norm(pr_vector)*np.linalg.norm(pr.w2v_features)) for pr_vector in list_vectors]
        ##if max(similar_scores) > 0.6:
        ##    print(reviewer)
        ##    print(pr.lda_text)
        ##    print(pr.id)
        ##    print(similar_scores.index(max(similar_scores)))
        ##    print((max(similar_scores)))
        ##    print(len(similar_scores))
        ##    print(len(reviewers_vectors[reviewer]))
        ##    print(reviewed_prs.iloc[similar_scores.index(max(similar_scores))].w2v_text)
        ##    print(reviewed_prs.iloc[similar_scores.index(max(similar_scores))].id)
        reviewers_semantic_sim.loc[pr.id, reviewer] = similar_scores
        #break
    #break

In [34]:
Counter(reviewers_semantic_sim['rchande'].isnull())

Counter({False: 8602})

In [211]:
for _, pr in test_data3.iterrows():
    for reviewer in test_data3.columns.values[10:-2]:
        reviewed_prs = train_data3[train_data3[reviewer]==1]
        list_vectors = list(reviewed_prs.w2v_features.values)
        #vector_features = normalize(pr.w2v_features[:,np.newaxis], axis=0).ravel() 
        #similar_scores = [np.dot(normalize(pr_vector[:,np.newaxis], axis=0).ravel(), vector_features) for pr_vector in reviewers_vectors[reviewer]]
        similar_scores = [np.dot(pr_vector, pr.w2v_features)/(np.linalg.norm(pr_vector)*np.linalg.norm(pr.w2v_features)) for pr_vector in list_vectors]
        #if max(similar_scores) > 0.6:
        #    print(reviewer)
        #    print(pr.lda_text)
        #    print(pr.id)
        #    print(similar_scores.index(max(similar_scores)))
        #    print((max(similar_scores)))
        #    print(len(similar_scores))
        #    print(len(reviewers_vectors[reviewer]))
        #    print(reviewed_prs.iloc[similar_scores.index(max(similar_scores))].w2v_text)
        #    print(reviewed_prs.iloc[similar_scores.index(max(similar_scores))].id)
        reviewers_semantic_sim.loc[pr.id, reviewer] = similar_scores
        #break
    #break

In [32]:
reviewers_semantic_sim[reviewers_semantic_sim['rchande'].isnull()].shape

(0, 54)

In [36]:
columns

array(['MattGertz', 'jcouv', 'jaredpar', 'mavasani',
       'VladimirReshetnikov', 'shyamnamboodiripad', 'dpoeschl',
       'ManishJayaswal', 'gafter', 'ghost', 'tannergooding', 'mattwar',
       'khyperia', 'paulvanbrenk', 'jmarolf', 'TyOverby', 'alrz',
       'AnthonyDGreen', 'sharwell', 'msJohnHamby', 'natidea', 'mmitche',
       'dotnet-bot', 'OmarTawfik', 'balajikris', 'KirillOsenkov',
       'MeiChin-Tsai', 'tmeschter', 'AArnott', 'panopticoncentral',
       'VSadov', 'dnfclas', 'amcasey', 'ivanbasov', 'tmat', 'stephentoub',
       'jinujoseph', 'KevinH-MS', 'agocke', 'genlu', 'srivatsn',
       'AlekseyTs', 'jasonmalinowski', 'DustinCampbell', 'rchande',
       'CyrusNajmabadi', 'AdamSpeight2008', 'brettfo', 'Pilchie',
       'davkean', 'cston', 'nguerrera', 'heejaechang', '333fred'],
      dtype=object)

In [40]:
Counter(reviewers_semantic_sim_max['rchande'].isnull())

Counter({False: 2373, True: 6229})

In [42]:
index = np.concatenate((train_data3.id.values, test_data3.id.values))
reviewers_semantic_sim_max = pd.DataFrame(index=index, columns=columns)
for row in index:
    for column in columns:
        similar_scores = reviewers_semantic_sim.loc[row, column]
        reviewers_semantic_sim_max.loc[row, column] = max(similar_scores)


In [43]:
reviewers_semantic_sim_sum = pd.DataFrame(index=index, columns=columns)
for row in index:
    for column in columns:
        similar_scores = reviewers_semantic_sim.loc[row, column]
        reviewers_semantic_sim_sum.loc[row, column] = sum(similar_scores)

In [44]:
reviewers_semantic_sim_average = pd.DataFrame(index=index, columns=columns)
for row in index:
    for column in columns:
        similar_scores = reviewers_semantic_sim.loc[row, column]
        reviewers_semantic_sim_average.loc[row, column] = sum(similar_scores)/len(similar_scores)

In [45]:
reviewers_semantic_sim_min = pd.DataFrame(index=index, columns=columns)
for row in index:
    for column in columns:
        similar_scores = reviewers_semantic_sim.loc[row, column]
        reviewers_semantic_sim_min.loc[row, column] = min(similar_scores)

In [46]:
reviewers_semantic_sim_max.to_csv('reviewers_semantic_sim_max.csv')
reviewers_semantic_sim_min.to_csv('reviewers_semantic_sim_min.csv')
reviewers_semantic_sim_sum.to_csv('reviewers_semantic_sim_sum.csv')
reviewers_semantic_sim_average.to_csv('reviewers_semantic_sim_average.csv')

# create train and test dataset

In [ ]:
import os
def save_array(array, filename):
    base_dir = 'D:\\reviewer recommendation\\LDA&w2v\\'
    np.save(os.path.join(base_dir, filename), array)

In [ ]:
# conduct train and test dataset
from sklearn.multiclass import OneVsRestClassifier
features_train = train_data3[['w2v_features']]
features_train_w2v = np.array(list(map(np.array, features_train.w2v_features)))

save_array(features_train_w2v, 'train_data.npy')

label_train = np.array(train_data3.loc[:, first_reviewer: last_reviewer])
save_array(label_train, 'train_label.npy')


features_test = test_data3[['w2v_features']]
features_test_w2v = np.array(list(map(np.array, features_test.w2v_features)))

save_array(features_test_w2v, 'test_data.npy')

label_test = np.array(test_data3.loc[:, first_reviewer: last_reviewer])
save_array(label_test, 'test_label.npy')

In [ ]:
print(label_test.shape)
test_data3.shape

label_test.sum(axis=0)==0

print(list(label_test.sum(axis=0)))

# construct the model

In [ ]:
from sklearn.neural_network import MLPClassifier
#parameters = {'activation':['logistic','tanh'],
#              'solver': ['sgd','adam'], 
#              'max_iter': [1000,2000,3000],
#              'alpha': 10.0 ** -np.arange(3, 7), 
#              'hidden_layer_sizes':[(128,64,32),(128,64)],
#              #'random_state':[0,1,2,3,4,5,6,7,8,9]
#             }
#
mlp = MLPClassifier(activation='logistic', solver='sgd', max_iter=10000, alpha=1e-03, hidden_layer_sizes=(8,))
mlp.fit(features_train_w2v, label_train)
# clf.predict()

In [ ]:
len(mlp.loss_curve_)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(mlp.loss_curve_)
ax.set_xlabel("iteration")
ax.set_ylabel("loss")
plt.show()

In [ ]:
np.save('score_test.npy',mlp.predict_proba(features_test_w2v))
np.save('df_probs_train.npy',mlp.predict_proba(features_train_w2v))

In [ ]:

mlp = OneVsRestClassifier(MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(32,16), max_iter = 100, random_state=1))
mlp.fit(features_train_w2v, label_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
parameters = {'n_estimators':[10, 50, 100, 500, 1000],
              'max_depth':[None, 3, 5, 10],
              'min_samples_split':[2,5,10]
             }
clf = GridSearchCV(RandomForestClassifier(), parameters, cv=3, scoring='precision_macro')
clf.fit(features_train_w2v, label_train)
print(clf.score(features_train_w2v, label_train))
print(clf.best_params_)
#model = OneVsRestClassifier(RandomForestClassifier(n_estimators=10000, max_depth=5, random_state=0, class_weight='balanced'))

In [ ]:
print("Best parameters set found on development set:")
#print()
print(clf.best_params_)
#print()
print("Grid scores on development set:")
#print()
means = clf.cv_results_['mean_train_score']
stds = clf.cv_results_['std_train_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
#print()

In [ ]:
df_probs = clf.predict_proba(features_test_w2v)
np.save('score_test.npy',np.array(df_probs))
print(df_probs)


# experiments

In [ ]:
from gensim.test.utils import get_tmpfile
vector_size = 150
train_corpus = list(map(lambda i:TaggedDocument(train_data3.iloc[i].tokenized_sentences,['doc%d' % i]), range(0,train_data3.shape[0])))

model = Doc2Vec(min_count=1, window=10, size=vector_size, sample=1e-4, negative=5, workers=8, epochs=200)
model.build_vocab(train_corpus)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

fname = get_tmpfile("D:\\reviewer recommendation\\LDA&w2v\\d2v_models\\my_doc2vec_model%s" % vector_size)

model.save(fname)
print('finish d2v model')


ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index('doc%d' % doc_id)
    ranks.append(rank)
    
#import collections
print(collections.Counter(ranks))
    

import pickle
with open('D:\\reviewer recommendation\\LDA&w2v\\store_ranks\\store_ranks%s.npy' % vector_size, 'wb') as writer:
    pickle.dump(ranks, writer)
    
#with open('store_ranks.npy', 'rb') as reader:
#    ranks3 = pickle.load(reader)
#collections.Counter(ranks3)

#model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
train_data3['w2v_features'] = list(map(lambda sentence:
                                      infer_w2v_features(model, sentence),
                                      train_data3.tokenized_sentences.values))
test_data3['w2v_features'] = list(map(lambda sentence:
                                      infer_w2v_features(model, sentence),
                                      test_data3.tokenized_sentences.values))

print('finish w2v features')
# conduct train and test dataset
from sklearn.multiclass import OneVsRestClassifier
features_train = train_data3[['w2v_features']]
features_train_w2v = np.array(list(map(np.array, features_train.w2v_features)))


label_train = np.array(train_data3.loc[:, first_reviewer: last_reviewer])
save_array(label_train, 'train_label.npy')


features_test = test_data3[['w2v_features']]
features_test_w2v = np.array(list(map(np.array, features_test.w2v_features)))


label_test = np.array(test_data3.loc[:, first_reviewer: last_reviewer])
save_array(label_test, 'test_label.npy')

from sklearn.neural_network import MLPClassifier
params = [
          {'solver': 'sgd', 'learning_rate': 'constant', 'alpha': 1e-3,'validation_fraction':0.3,
           'learning_rate_init': 0.005, 'max_iter': 20000,'hidden_layer_sizes': (32)},
    
          {'solver': 'sgd', 'learning_rate': 'constant', 'alpha': 1e-3,'validation_fraction':0.3,
           'learning_rate_init': 0.01, 'max_iter': 10000,'hidden_layer_sizes': (32,16)},
    
          {'solver': 'sgd', 'learning_rate': 'constant', 'alpha': 1e-3,'validation_fraction':0.3,
           'learning_rate_init': 0.01, 'max_iter': 10000,'hidden_layer_sizes': (64,32)},
    
          #{'solver': 'sgd', 'learning_rate': 'constant', 'alpha': 1e-3,'validation_fraction':0.3,
          # 'learning_rate_init': 0.01, 'max_iter': 10000,'hidden_layer_sizes': (16, 8)},
    
          {'solver': 'sgd', 'learning_rate': 'constant', 'alpha': 1e-3,'validation_fraction':0.3,
           'learning_rate_init': 0.001, 'max_iter': 10000,'hidden_layer_sizes': (8,)},
    
          {'solver': 'sgd', 'learning_rate': 'constant', 'alpha': 1e-3,'validation_fraction':0.3,
           'learning_rate_init': 0.001, 'max_iter': 10000,'hidden_layer_sizes': (16,)}
    
          #{'solver': 'sgd', 'learning_rate': 'constant', 'alpha': 1e-3,'validation_fraction':0.3,
          # 'learning_rate_init': 0.001, 'max_iter': 10000,'hidden_layer_sizes': (4,)}
          #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
          # 'learning_rate_init': 0.001, 'max_iter': 10000, 'hidden_layer_sizes': (8,)},
          #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
          #'learning_rate_init': 0.001, 'max_iter': 10000, 'hidden_layer_sizes': (4,)},
          #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
          # 'learning_rate_init': 0.001, 'max_iter': 10000, 'hidden_layer_sizes': (8-4,)},
          ]
#labels = ["hidden64", "hidden32", "hidden16", "hidden8", "hidden4", "hidden8-4"]
labels = ["hidden32", "hidden32-16", "hidden64-32", "hidden8", "hidden16" ]
mlps = []
for label, param in zip(labels, params):
    print("training: %s" % label)
    mlp = MLPClassifier(verbose=0, random_state=1, n_iter_no_change=100, **param)
    mlp.fit(features_train_w2v, label_train)
    np.save('D:\\reviewer recommendation\\LDA&w2v\\score_test\\score_test%s_%s.npy' % (vector_size, label), mlp.predict_proba(features_test_w2v))
    np.save('D:\\reviewer recommendation\\LDA&w2v\\score_train\\score_train%s_%s.npy' % (vector_size, label), mlp.predict_proba(features_train_w2v))
    mlps.append(mlp)
   #print("Training set score: %f" % mlp.score(X, y))
   #print("Training set loss: %f" % mlp.loss_)
    
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("iteration")
ax.set_ylabel("loss")
for mlp, label in zip(mlps, labels):
        ax.plot(mlp.loss_curve_, label=label)
plt.legend(loc=1)
plt.show()

#mlp = MLPClassifier(activation='logistic', solver='sgd', max_iter=10000, alpha=1e-03, hidden_layer_sizes=(8,))
#mlp.fit(features_train_w2v, label_train)
# clf.predict()



In [ ]:

with open('store_ranks.npy', 'rb') as reader:
    ranks3 = pickle.load(reader)
collections.Counter(ranks3)

# visulization experiment result

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.utils.fixes import signature
def draw_precision_recall_curve(score, label):
    precision = dict()
    recall = dict()
    average_precision = dict()

    Y_test = np.load(label)
    Y_score = np.load(score)
    print(Y_test.shape)
    print(Y_score.shape)
    precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(), Y_score.ravel())
    average_precision["micro"] = average_precision_score(Y_test, Y_score,
                                                     average="micro")
    print('Average precision score, micro-averaged over all classes: {0:0.2f}' .format(average_precision["micro"]))
    plt.figure()
    plt.step(recall['micro'], precision['micro'], color='b', alpha=0.2, where='post')
    plt.step(recall['micro'], precision['micro'], color='g', alpha=0.2, where='post')
    step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
    plt.fill_between(recall["micro"], precision["micro"], alpha=0.2, color='b', **step_kwargs)
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title( 'Average precision score, micro-averaged over all classes: AP={0:0.2f}' .format(average_precision["micro"]))

In [ ]:
# download the df_probs_train.npy and *_test.npy files from server
draw_precision_recall_curve('df_probs_train.npy', 'train_label.npy')
draw_precision_recall_curve('score_test.npy', 'test_label.npy')

# no longer needed

In [ ]:
import json

best_params = {'n_estimators':100, 'max_depth':3, 'min_samples_split':2}
cv_results_ = {
'split0_test_score'  : [0.80, 0.70, 0.80, 0.93],
'split1_test_score'  : [0.82, 0.50, 0.70, 0.78],
'mean_test_score'    : [0.81, 0.60, 0.75, 0.85],
'std_test_score'     : [0.01, 0.10, 0.05, 0.08],
'rank_test_score'    : [2, 4, 3, 1],
'split0_train_score' : [0.80, 0.92, 0.70, 0.93],
'split1_train_score' : [0.82, 0.55, 0.70, 0.87],
'mean_train_score'   : [0.81, 0.74, 0.70, 0.90],
'std_train_score'    : [0.01, 0.19, 0.00, 0.03],
'mean_fit_time'      : [0.73, 0.63, 0.43, 0.49],
'std_fit_time'       : [0.01, 0.02, 0.01, 0.01],
'mean_score_time'    : [0.01, 0.06, 0.04, 0.04],
'std_score_time'     : [0.00, 0.00, 0.00, 0.01],
'params'             : [{'kernel': 'poly', 'degree': 2}],
}
means = cv_results_['mean_test_score']
stds = cv_results_['std_test_score']
for mean, std, params in zip(means, stds, cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

with open('gridsearch.txt','w') as file:
    file.write('best params\n')
    file.write(json.dumps(best_params))
    file.write('\nmean_test_score\n')
    file.write(json.dumps(cv_results_['mean_test_score']))
    file.write('\nstd_test_score\n')
    file.write(json.dumps(cv_results_['std_test_score']))
    file.write('\nmean_train_score\n')
    file.write(json.dumps(cv_results_['mean_train_score']))
    file.write('\nstd_train_score\n')
    file.write(json.dumps(cv_results_['std_train_score']))
    

In [ ]:
label_train_resampled[label_train_resampled.sum(axis=1)!=1]

In [ ]:
# train the model and test its performance
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import average_precision_score
model.fit(features_train_w2v, label_train)

#for x, y in zip(features_test_w2v[0:20,], label_test[0:20,]):
#    print(x)
#    print(y)
#    y_pred = model.predict(x.reshape(1, -1))
#    print(y_pred[0])
#    precision = precision_score(y, y_pred[0])
#    recall = recall_score(y, y_pred[0])
#    print(precision)
#    print(recall)
df_probs = model.predict_proba(features_test_w2v)
print(df_probs.shape)
print(average_precision_score(label_test, df_probs, average='weighted'))

for i in range(1,10):
    result = pd.DataFrame({'Precision' :[], 'Recall' :[]}, dtype=np.int64)
    
    
    for probs, y in zip(df_probs, label_test):
        y_pred = np.zeros_like(probs, dtype=np.int64)
    
        y_pred[probs.argsort()[-i:]] = 1
        precision = precision_score(y, y_pred)
        recall = recall_score(y, y_pred)
        #print("Training Precision = ", precision)
        #print("Training Recall = ", recall)
        result = result.append({'Precision':precision, 'Recall':recall}, ignore_index=True)
    filename = 'D:\\reviewer recommendation\\LDA&w2v\\result\\2080split%dt.csv' % i
    result.to_csv(filename)
    
df_probs = model.predict_proba(features_train_w2v)
print(df_probs.shape)
print(average_precision_score(label_train, df_probs, average='weighted'))

for i in range(1,10):
    result = pd.DataFrame({'Precision' :[], 'Recall' :[]}, dtype=np.int64)
    
    
    for probs, y in zip(df_probs, label_train):
        y_pred = np.zeros_like(probs, dtype=np.int64)
    
        y_pred[probs.argsort()[-i:]] = 1
        precision = precision_score(y, y_pred)
        recall = recall_score(y, y_pred)
        #print("Training Precision = ", precision)
        #print("Training Recall = ", recall)
        result = result.append({'Precision':precision, 'Recall':recall}, ignore_index=True)
    filename = 'D:\\reviewer recommendation\\LDA&w2v\\result\\2080split%d.csv' % i
    result.to_csv(filename)

In [ ]:
np.save('numpy.npy', df_probs)
d = np.load('numpy.npy')

In [ ]:
def show_result(i, n=1):
    
    column_names = test_data3.loc[:,first_reviewer:last_reviewer].columns.values
    
    labels = np.array(test_data3.iloc[i].loc[first_reviewer:last_reviewer])
    print('true:' + column_names[labels==1])

    print('prediction:' + column_names[df_probs[i].argsort()[-n:]])

In [ ]:
for i in range(0,20):
    show_result(i,1)

In [ ]:
print(train_data3[train_data3['jaredpar']==1].shape)
print(train_data3[train_data3['jasonmalinowski']==1].shape)
print(train_data3[train_data3['Pilchie']==1].shape)
print(train_data3[train_data3['CyrusNajmabadi']==1].shape)
print(train_data3[train_data3['VSadov']==1].shape)
print(train_data3[train_data3['cston']==1].shape)

In [ ]:
Precision = []
Recall = []
for i in range(1,10):
    filename = 'D:\\reviewer recommendation\\LDA&w2v\\result\\2080split%d.csv' % i

    result = pd.read_csv(filename)
    #del result['Unnamed: 0']
    Precision.append(result['Precision'].mean())
    Recall.append(result['Recall'].mean())

In [ ]:
def draw_line(data, label):
    plt.plot(data, '-', lw=2, label=label)
    plt.xlabel('# of reviewers')
    plt.ylim(0,1)
    plt.ylabel('average value')
    plt.legend(prop={'size': 12}, loc='upper right')

In [ ]:
draw_line(Precision, 'Precision')
draw_line(Recall, 'Recall')

In [ ]:
f = plt.figure()
plt.plot(range(10), range(10), "o")
plt.show()

f.savefig("foo.pdf", bbox_inches='tight')

# balance the data

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
#features_train_w2v_resampled, label_train_resampled = ros.fit_resample(features_train_w2v, label_train)
features_train_w2v_resampled, label_train_resampled = SMOTE().fit_resample(features_train_w2v, label_train)
#features_train_w2v_resampled, label_train_resampled = ADASYN().fit_resample(features_train_w2v, label_train)


In [ ]:
label_train_resampled.shape

In [ ]:
list_dict = []
for i in range(0, label_train.shape[1]):
    count_dict = Counter(label_train[:,i])
    count_0 = count_dict[0]
    count_1 = count_dict[1]
    count_dict = dict()
    count_dict[0] = count_1
    count_dict[1] = count_0
    list_dict.append(count_dict)